In [ ]:
# import models and solvers
from centroidal_plus_double_integrator_kinematics_acados_solver import CentroidalPlusLegKinematicsAcadosSolver
from centroidal_plus_double_integrator_kinematics_casadi_model  import CentroidalPlusLegKinematicsCasadiModel
import conf_solo12_bound_step_adjustment_full_kinematics_mpc as conf
from robot_properties_solo.solo12wrapper import Solo12Config
from wholebody_croccodyl_solver import WholeBodyDDPSolver
from wholebody_croccodyl_model import WholeBodyModel
import pinocchio as pin
import numpy as np
import utils
import sys

# DDP warm-start
wbd_model = WholeBodyModel(conf)
ddp_planner = WholeBodyDDPSolver(wbd_model, MPC=False, WARM_START=False)
ddp_planner.solve()
ddp_sol = ddp_planner.get_solution_trajectories()
centroidal_warmstart = ddp_sol['centroidal']
q_warmstart = ddp_sol['jointPos']
qdot_warmstart = ddp_sol['jointVel']
x_warmstart = []
u_warmstart = []
rmodel, rdata = conf.rmodel, conf.rdata
for k in range(len(centroidal_warmstart)):
    x_warmstart.append(
        np.concatenate(
            [centroidal_warmstart[k],
             q_warmstart[k], 
             qdot_warmstart[k]]
            )
        )
    u_warmstart.append(np.concatenate([np.zeros(30)]))

# create robot
robot = Solo12Config.buildRobotWrapper()
# load robot in meshcat viewer
viz = pin.visualize.MeshcatVisualizer(
robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print(err)
    sys.exit(0)
viz.loadViewerModel()
# add contact surfaces
s = 0.5*conf.step_adjustment_bound
for i, contacts in enumerate(conf.contact_sequence):
    for contact_idx, contact in enumerate(contacts):
        if contact.ACTIVE:
            t = contact.pose.translation
            # debris box
            if contact.CONTACT == 'FR' or contact.CONTACT == 'FL':
                utils.addViewerBox(
                    viz, 'world/debris'+str(i)+str(contact_idx), 
                    2*s, 2*s, 0., [1., .2, .2, .5]
                    )
            if contact.CONTACT == 'HR' or contact.CONTACT == 'HL':
                utils.addViewerBox(
                    viz, 'world/debris'+str(i)+str(contact_idx),
                    2*s, 2*s, 0., [.2, .2, 1., .5]
                    )
            utils.applyViewerConfiguration(
                viz, 'world/debris'+str(i)+str(contact_idx), 
                [t[0], t[1], t[2]-0.017, 1, 0, 0, 0]
                )
            utils.applyViewerConfiguration(
                viz, 'world/debris_center'+str(i)+str(contact_idx), 
                [t[0], t[1], t[2]-0.017, 1, 0, 0, 0]
                ) 
# visualize DDP warm-start
for q_warmstart_k in q_warmstart:
    for i in range(10+int(conf.dt/conf.dt_ctrl)):
        viz.display(q_warmstart_k)

# nominal traj-opt
model_nom = CentroidalPlusLegKinematicsCasadiModel(conf, STOCHASTIC_OCP=False)
solver_nom = CentroidalPlusLegKinematicsAcadosSolver(
    model_nom, x_warmstart, u_warmstart, MPC=True
)
x_nom, u_nom, lqr_gains = solver_nom.solve()

# stochastic traj-opt
model_stoch = CentroidalPlusLegKinematicsCasadiModel(conf, STOCHASTIC_OCP=True)
solver_stoch = CentroidalPlusLegKinematicsAcadosSolver(
    model_stoch, x_warmstart, u_warmstart, MPC=True
)
x_stoch, u_stoch, lqr_gains = solver_stoch.solve()

In [ ]:
from centroidal_plus_double_integrator_kinematics_acados_simulator import CentroidalPlusLegKinematicsAcadosSimulator
# run monte-carlo simulations
nb_sims = 100
simulator_nom = CentroidalPlusLegKinematicsAcadosSimulator(model_nom, solver_nom)
# sample addtitive disturbances and pass them to both simulators
w_total = np.zeros((nb_sims, conf.N-1, x_nom.shape[1]))
for sim in range(nb_sims):
    for time_idx in range(conf.N-1):
        w_total[sim, time_idx, :] = simulator_nom.sample_addtitive_uncertainties() 

# simulate nominal MPC
x_sim_nom = simulator_nom.simulate(x_nom[0, :], u_nom, lqr_gains, np.copy(w_total), nb_sims=nb_sims, WITH_DISTURBANCES=True)
violations_nom, ee_positions_nom = simulator_nom.count_constraint_violations(x_sim_nom, WITH_VISUALIZATION=False)
print("number of contact location constraint violations for nominal MPC :", violations_nom)

# simulate stochastic MPC
simulator_stoch = CentroidalPlusLegKinematicsAcadosSimulator(model_stoch, solver_stoch)
x_sim_stoch = simulator_stoch.simulate(x_stoch[0, :], u_stoch, lqr_gains, np.copy(w_total), nb_sims=nb_sims, WITH_DISTURBANCES=True)
violations_stoch, ee_positions_stoch = simulator_stoch.count_constraint_violations(x_sim_stoch, WITH_VISUALIZATION=False)
print("number of contact location constraint violations for stochastic MPC :", violations_stoch)

In [ ]:
from robot_properties_solo.solo12wrapper import Solo12Config
import pinocchio as pin
import utils

# create robot
robot = Solo12Config.buildRobotWrapper()
# load robot in meshcat viewer
viz = pin.visualize.MeshcatVisualizer(
robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print(err)
    sys.exit(0)
viz.loadViewerModel()
# add nominal contact surfaces
s = simulator_nom.debris_half_size
for i, contacts in enumerate(simulator_nom.contact_sequence):
    for contact_idx, contact in enumerate(contacts):
        if contact.ACTIVE:
            t = contact.pose.translation
            # debris box
            if contact.CONTACT == 'FR' or contact.CONTACT == 'FL':
                utils.addViewerBox(
                    viz, 'world/debris'+str(i)+str(contact_idx), 
                    2*s, 2*s, 0., [1., .2, .2, .5]
                    )
            if contact.CONTACT == 'HR' or contact.CONTACT == 'HL':
                utils.addViewerBox(
                    viz, 'world/debris'+str(i)+str(contact_idx),
                    2*s, 2*s, 0., [.2, .2, 1., .5]
                    )
            utils.applyViewerConfiguration(
                viz, 'world/debris'+str(i)+str(contact_idx), 
                [t[0], t[1], t[2]-0.017, 1, 0, 0, 0]
                )
            utils.applyViewerConfiguration(
                viz, 'world/debris_center'+str(i)+str(contact_idx), 
                [t[0], t[1], t[2]-0.017, 1, 0, 0, 0]
                )
                
ee_pos_total_nom = []
ee_pos_total_stoch = []
# visualize end-effector trajectories (TODO: visualize trajectories in mujoco with an army of robots)
for sim in range(nb_sims):
    FL_ee_nom = []
    FR_ee_nom = []
    HL_ee_nom = []
    HR_ee_nom = []
    FL_ee_stoch = []
    FR_ee_stoch = []
    HL_ee_stoch = []
    HR_ee_stoch = []
    for traj_length_nom in range(len(ee_positions_nom[sim])):
        FL_ee_nom += [ee_positions_nom[sim][traj_length_nom][0]]
        FR_ee_nom += [ee_positions_nom[sim][traj_length_nom][1]]
        HL_ee_nom += [ee_positions_nom[sim][traj_length_nom][2]]
        HR_ee_nom += [ee_positions_nom[sim][traj_length_nom][3]]
    for traj_length_stoch in range(len(ee_positions_stoch[sim])):
        FL_ee_stoch += [ee_positions_stoch[sim][traj_length_stoch][0]]
        FR_ee_stoch += [ee_positions_stoch[sim][traj_length_stoch][1]]
        HL_ee_stoch += [ee_positions_stoch[sim][traj_length_stoch][2]]
        HR_ee_stoch += [ee_positions_stoch[sim][traj_length_stoch][3]]
    # plot end-effectors trajectories
    utils.addLineSegment(viz, 'ee_trajectory_nom_FL'+str(sim), np.array(FL_ee_nom).astype(np.float32).T, [1,1,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_nom_FR'+str(sim), np.array(FR_ee_nom).astype(np.float32).T, [1,1,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_nom_HL'+str(sim), np.array(HL_ee_nom).astype(np.float32).T, [1,1,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_nom_HR'+str(sim), np.array(HR_ee_nom).astype(np.float32).T, [1,1,0,1])    
    utils.addLineSegment(viz, 'ee_trajectory_stoch_FL'+str(sim), np.array(FL_ee_stoch).astype(np.float32).T, [1,0,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_stoch_FR'+str(sim), np.array(FR_ee_stoch).astype(np.float32).T, [1,0,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_stoch_HL'+str(sim), np.array(HL_ee_stoch).astype(np.float32).T, [1,0,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_stoch_HR'+str(sim), np.array(HR_ee_stoch).astype(np.float32).T, [1,0,0,1])

In [ ]:
# compute the norm of the difference between the closed-loop contact location and the center of contact surface
contacts_logic_N = simulator_nom.contact_data['contacts_logic']
contacts_position_N = simulator_nom.contact_data['contacts_position'] 
contact_location_nom_total   = [] 
contact_location_stoch_total = [] 
contact_surface_location_nom_total = []
contact_surface_location_stoch_total = []
norm_contact_location_deviation_nom_total = []
norm_contact_location_deviation_stoch_total = []
for sim in range(nb_sims):
    contact_location_nom_per_traj = []
    contact_location_stoch_per_traj = []
    contact_surface_location_nom_per_traj = []
    contact_surface_location_stoch_per_traj = []
    norm_contact_location_deviation_nom_per_traj = []
    norm_contact_location_deviation_stoch_per_traj = []
    for time_idx in range(len(ee_positions_stoch[sim])):
        contact_location_nom_k = []
        contact_location_stoch_k = []
        contact_surface_location_nom_k = []
        contact_surface_location_stoch_k = []
        for contact_idx in range(len(conf.ee_frame_names)):
            # FL
            if contact_idx==0:
                CURR_IN_CONTACT = contacts_logic_N[time_idx][1]
                contact_surface_location_per_foot = contacts_position_N[time_idx][3:6]
            # FR
            elif contact_idx==1:
                CURR_IN_CONTACT = contacts_logic_N[time_idx][0]
                contact_surface_location_per_foot = contacts_position_N[time_idx][0:3]
            # HL
            elif contact_idx==2:
                CURR_IN_CONTACT = contacts_logic_N[time_idx][3]
                contact_surface_location_per_foot = contacts_position_N[time_idx][9:12]
            # HR
            elif contact_idx==3:
                CURR_IN_CONTACT = contacts_logic_N[time_idx][2]
                contact_surface_location_per_foot = contacts_position_N[time_idx][6:9]
            # compute contact location deviation only for the activated contacts
            if CURR_IN_CONTACT:
                contact_surface_location_stoch_k += [contact_surface_location_per_foot]
                contact_location_stoch_k += [ee_positions_stoch[sim][time_idx][contact_idx]]
                if time_idx < len(ee_positions_nom[sim]):
                    contact_surface_location_nom_k += [contact_surface_location_per_foot]
                    contact_location_nom_k += [ee_positions_nom[sim][time_idx][contact_idx]] 
        # since some trajectories fail before resuming till the end in the nominal case
        if time_idx < len(ee_positions_nom[sim]):
            contact_location_nom_per_traj += [contact_location_nom_k]
            contact_surface_location_nom_per_traj += [contact_surface_location_nom_k]
            # compute the contact location at the time of landing
#             if CURR_IN_CONTACT:
            norm_contact_location_deviation_nom_per_traj.append(
                np.linalg.norm(np.asarray(contact_location_nom_k) - np.asarray(contact_surface_location_nom_k))
            )
#             else:
#             norm_contact_location_deviation_nom_per_traj.append(0.)
        contact_location_stoch_per_traj += [contact_location_stoch_k]
        contact_surface_location_stoch_per_traj += [contact_surface_location_stoch_k]
#         if CURR_IN_CONTACT:
        norm_contact_location_deviation_stoch_per_traj.append(
                    np.linalg.norm(np.asarray(contact_location_stoch_k) - np.asarray(contact_surface_location_stoch_k))
                )
#         else:
#         norm_contact_location_deviation_stoch_per_traj.append(0.)
    contact_location_nom_total += [contact_location_nom_per_traj]
    contact_location_stoch_total += [contact_location_stoch_per_traj]
    contact_surface_location_nom_total += [contact_surface_location_nom_per_traj]
    contact_surface_location_stoch_total += [contact_surface_location_stoch_per_traj]
    norm_contact_location_deviation_nom_total += [norm_contact_location_deviation_nom_per_traj]
    norm_contact_location_deviation_stoch_total += [norm_contact_location_deviation_stoch_per_traj]

In [ ]:
import matplotlib.pylab as plt

# compute statistics
mean_nom_total = np.zeros(conf.N)
std_nom_total = np.zeros(conf.N)
mean_stoch_total = np.zeros(conf.N)
std_stoch_total = np.zeros(conf.N)
for traj_idx in range(conf.N):
    nb_samples_nom = 0.
    nb_samples_stoch = 0.
    samples_nom_k = []
    samples_stoch_k = []
    for sim in range(nb_sims):
        if traj_idx < len(norm_contact_location_deviation_nom_total[sim]):
            mean_nom_total[traj_idx] += norm_contact_location_deviation_nom_total[sim][traj_idx]
            samples_nom_k += [norm_contact_location_deviation_nom_total[sim][traj_idx]]
            nb_samples_nom += 1
        mean_stoch_total[traj_idx] += norm_contact_location_deviation_stoch_total[sim][traj_idx]
        samples_stoch_k += [norm_contact_location_deviation_stoch_total[sim][traj_idx]]
        nb_samples_stoch += 1
    mean_nom_total[traj_idx] = np.mean(samples_nom_k)
    std_nom_total[traj_idx] = np.std(samples_nom_k)
    mean_stoch_total[traj_idx] = np.mean(samples_stoch_k)
    std_stoch_total[traj_idx] = np.std(samples_stoch_k)

# plot the mean over the horizon length 
fig, ax = plt.subplots(1, 1, sharex=True) 
for sim in range(nb_sims):
    time_nom   = np.arange(0, np.round((len(norm_contact_location_deviation_nom_total[sim]))*conf.dt, 2), conf.dt)
    time_stoch = np.arange(0, np.round((len(norm_contact_location_deviation_stoch_total[sim]))*conf.dt, 2), conf.dt)
ax.plot(time_stoch, mean_stoch_total, color='green')
ax.fill_between(time_stoch, mean_stoch_total+2*std_stoch_total, mean_stoch_total-2*std_stoch_total, color='green', alpha=0.1)
ax.plot(time_stoch, mean_nom_total, color='blue')
ax.fill_between(time_stoch, mean_nom_total+2*std_nom_total, mean_nom_total-2*std_nom_total, color='blue', alpha=0.1)
ax.set_xlabel('Time (s)', fontsize=14)
ax.set_ylabel('norm of contact locations deviations', fontsize=14)
fig.suptitle('bounding motion')
